In [7]:
import re
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [20]:
with open('./results/lab1.json') as lab1_file:
    data = json.load(lab1_file)
tokens_chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

tokens_chain

[['W', 0, 'program'],
 ['I', 0, 'simple'],
 ['R', 4, ';'],
 ['W', 1, 'var'],
 ['I', 1, 'a'],
 ['R', 3, ':'],
 ['W', 3, 'integer'],
 ['R', 4, ';'],
 ['I', 2, 's'],
 ['R', 3, ':'],
 ['W', 5, 'string'],
 ['R', 4, ';'],
 ['W', 11, 'begin'],
 ['I', 1, 'a'],
 ['W', 12, ':='],
 ['R', 5, '('],
 ['C', 0, '5'],
 ['O', 0, '+'],
 ['C', 1, '3'],
 ['R', 6, ')'],
 ['O', 3, '/'],
 ['C', 2, '1'],
 ['R', 4, ';'],
 ['W', 14, 'if'],
 ['I', 1, 'a'],
 ['O', 6, '>'],
 ['C', 2, '1'],
 ['W', 15, 'then'],
 ['I', 2, 's'],
 ['W', 12, ':='],
 ['C', 3, 'a greater 1'],
 ['W', 16, 'else'],
 ['I', 2, 's'],
 ['W', 12, ':='],
 ['C', 4, 'a less or equal 1'],
 ['R', 4, ';'],
 ['W', 18, 'end.']]

In [16]:
class BasicGenerator:
    program_on_basic = ''
    tokens_chain = []
    
    states = ['S']
    
    pascal_tables = {
        'W': ['program', 'var', 'const', 'integer', 'real', 'string', 'label',
                 'array', 'of', 'procedure', 'function',
                 'begin', ':=', 'goto', 'if', 'then', 'else', 'end', 'end.'],
        'O': ['+', '-', '*', '/', '^', '<', '>', '=', '<>', '<=', '>='],
        'R': [' ', ',', '..', ':', ';', '(', ')', '[', ']', '{', '}', '\''],
    }
    basic_tables = {
        'W': ['', 'var', 'const', 'integer', 'real', 'string', 'label',
                     'array', 'of', 'GOSUB', 'GOSUB',
                     'begin', '=', 'goto', 'IF', 'THEN', 'ELSE', 'END', 'END'],
        'O': ['+', '-', '*', '/', '^', '<', '>', '=', '<>', '<=', '>='],
        'R': [' ', ',', '..', ':', '\n', '(', ')', '[', ']', '{', '}', '\''],
    }
    
    var_pool = []
    array_interval = ''
    
    def __init__(self, tokens_chain, tables):
        self.tokens_chain = tokens_chain
        self.pascal_tables = tables
        
        self.program_on_basic = ''
        self.array_interval = ''
        self.var_pool = []
        self.states = ['S']
    
    def value(self, token):
        if token[0] == 'C':
            return self.pascal_tables[token[0]][token[1]]['value']
        if token[0] == 'I':
            return self.pascal_tables[token[0]][token[1]]['name']
        return self.pascal_tables[token[0]][token[1]]
    
    def back(self, token):
        self.tokens_chain = [token] + self.tokens_chain
        
    
    def nextStep(self):
        try:
            token = self.tokens_chain[0]
            self.tokens_chain = self.tokens_chain[1:]
        except:
            token = []
            return self.program_on_basic
        
        if self.states[-1] == 'S':
            
            if self.value(token) == 'program':
                self.states.append('program')

            if self.value(token) == 'var':
                self.states.append('var')
                
            if self.value(token) == 'label':
                self.states.append('label')
                
            if self.value(token) == 'const':
                self.states.append('const')
                
            if self.value(token) == 'begin':
                pass
                
            return self.nextStep()
            
        if self.states[-1] == 'var':
            
            if token[0] == 'I':
                self.var_pool.append(self.value(token))
                
            if self.value(token) in ['integer', 'real', 'string']:
                self.var_pool = []
            
            if self.value(token) == '[':
                self.states.append('var [')
            
            if token[0] == 'W' and (not self.value(token) in ['array', 'of', 'integer', 'real', 'string']):
                self.states.pop()
                self.back(token)
                
            return self.nextStep()
        
        if self.states[-1] == 'var [':
            
            if self.value(token) == ']':
                self.states.append('var []')
            else:
                self.array_interval += self.value(token)
            
            return self.nextStep()
        
        if self.states[-1] == 'var []':
            
            if self.value(token) in ['integer', 'real', 'string']:
                var_type = self.value(token)
                
                new_code = 'DIM '
                for var in self.var_pool:
                    new_code += var
                    if var_type == 'string':
                        new_code += '$'
                    if var_type == 'integer':
                        new_code += '%'
                    if var_type == 'real':
                        new_code += '!'
                        
                    new_code += '(' + self.array_interval.replace('..', ':').replace(',', ', ') + '), '
                new_code = new_code[:-2]
                
                self.array_interval = ''
                self.var_pool = []
                
                self.program_on_basic += new_code + '\n'
                print(new_code)
                self.states.pop()
                
            return self.nextStep()         
        
        
        if self.states[-1] == 'const':
            if token[0] == 'W':
                self.back(token)
                self.state = 'S'
            return self.nextStep()
        
        if self.states[-1] in ['program', 'label']:
            if self.value(token) == ';':
                self.states.pop()
            return self.nextStep()
            
            
        return self.program_on_basic
            

In [19]:
result = BasicGenerator(
    tokens_chain,
    {
        'W': service_words,
        'O': operations,
        'R': separators,
        'C': constants,
        'I': identifiers,
    }
)
output = result.nextStep()
print()
print(output)